In [1]:
from pyspark.context import SparkContext
from pyspark import SparkConf,sql
from pyspark.sql.session import SparkSession
import findspark
import pymysql
import pymysql.cursors
import pandas as pd

In [2]:
findspark.init()
findspark.find()

'C:\\spark\\spark-3.5.0-bin-hadoop3'

In [48]:
product_path='D:/PROJECTS/FASHIONPASS/Database/product.csv'
tagcloud_path='D:/PROJECTS/FASHIONPASS/Database/tag_cloud.csv'
collection_category_tag_path='D:/PROJECTS/FASHIONPASS/Database/collection_category_tag.csv'
product_tag_path='D:/PROJECTS/FASHIONPASS/Database/product_tag.csv'
features_path='D:/PROJECTS/FASHIONPASS/Database/spars_dense_features.csv'
collection_category_path='D:/PROJECTS/FASHIONPASS/Database/collection_category.csv'
clothing_collection_path='D:/PROJECTS/FASHIONPASS/Database/cloting_collection_category_tag_v1_10_1_2023.csv'#cloting_collection_category_tag_10_1_2023.csv'#D:/PROJECTS/FASHIONPASS/Database/cloting_collection_category.csv'

In [49]:
spark = SparkSession.builder \
    .appName("PySpark MySQL Example") \
    .config("spark.driver.extraClassPath", "C:/spark/spark-3.5.0-bin-hadoop3/jars/mysql-connector-java-5.1.46.jar") \
    .getOrCreate()



In [50]:
Product_df=spark.read.format('csv').option('header',True) \
.load(product_path)

Tagcloud_df=spark.read.format('csv').option('header',True) \
.load(tagcloud_path)


Collection_Category_tag_df=spark.read.format('csv').option('header',True) \
.load(collection_category_tag_path)

Product_tag_df=spark.read.format('csv').option('header',True) \
.load(product_tag_path)


features_df=spark.read.format('csv').option('header',True) \
.load(features_path)


Collection_Category_df=spark.read.format('csv').option('header',True)\
.load(collection_category_path)
clothing_collection_df=spark.read.format('csv').option('header',True)\
.load(clothing_collection_path)

In [51]:
def duplicate_col(data):
  df_cols = data.columns
  duplicate_col_index = [idx for idx,
  val in enumerate(df_cols) if val in df_cols[:idx]]
 
# Create a new list by renaming duplicate 
# columns by adding prefix '_duplicate_'+index
  for i in duplicate_col_index:
    df_cols[i] = df_cols[i] + '_duplicate_'+ str(i)
 
# Rename the duplicate columns in data frame
  data = data.toDF(*df_cols)
 
# Display the data frame
  return data

In [52]:
def string_column(data_df):
    for key,value in data_df.items():
        data_df[key]=[key if (v != '' and int(v) >0) else '' for v in value]
        # print([ key if  v != '0' else '' for v in value ])
        # print([key if (v != '' and int(v) >0) else '' for v in value])
    return data_df

In [53]:
clothing_tag=[ i[0] for i in  clothing_collection_df.select('tag_id').collect()]

In [54]:
product_product_tag_df=Product_tag_df.join(Tagcloud_df,Product_tag_df.tag_id==Tagcloud_df.id,'left')

In [55]:
product_product_tag_cloting_df=product_product_tag_df.filter(product_product_tag_df['id'].isin(clothing_tag))

In [56]:
product_product_tag_cloting_df=product_product_tag_cloting_df.join(clothing_collection_df,clothing_collection_df.tag_id==product_product_tag_cloting_df.tag_id,'left')

In [57]:
product_product_tag_cloting_df.show()

+---+----------+------+---+---+--------+-------------------+----------+-------------------+----------+---+----+--------------------+--------+------+
|_c0|product_id|tag_id|_c0| id|tag_name|         updated_at|updated_by|         created_at|created_by|_c0|  id|                name|tag_name|tag_id|
+---+----------+------+---+---+--------+-------------------+----------+-------------------+----------+---+----+--------------------+--------+------+
|  1|         1|    50| 29| 50| bottoms|2023-08-22 12:32:04|       6.0|2020-07-13 04:54:08|         0|335|  60|            Swimwear| bottoms|    50|
|  1|         1|    50| 29| 50| bottoms|2023-08-22 12:32:04|       6.0|2020-07-13 04:54:08|         0|297|  57|              Skirts| bottoms|    50|
|  1|         1|    50| 29| 50| bottoms|2023-08-22 12:32:04|       6.0|2020-07-13 04:54:08|         0|285|  56|              Shorts| bottoms|    50|
|  1|         1|    50| 29| 50| bottoms|2023-08-22 12:32:04|       6.0|2020-07-13 04:54:08|         0|260|

In [58]:
product_product_tag_cloting_df=duplicate_col(product_product_tag_cloting_df)

In [59]:
product_product_tag_cloting_df['product_id','tag_name','name'].show()

+----------+--------+--------------------+
|product_id|tag_name|                name|
+----------+--------+--------------------+
|         1| bottoms|            Swimwear|
|         1| bottoms|              Skirts|
|         1| bottoms|              Shorts|
|         1| bottoms|               Pants|
|         1| bottoms|Activewear & Loun...|
|         1|    work|                Tops|
|         1|    work|                Tops|
|         1|    work|                Work|
|         1|    work|                Work|
|         1|    work|                Work|
|         1|    work|                Tops|
|         1|    work|           Occasions|
|         1|   black|               Black|
|         1|   black|               Black|
|         1|   black|               Black|
|         1|   black|               Black|
|         1|   black|               Black|
|         1|   black|               Pants|
|         1|   black|           Jumpsuits|
|         1|   black|       Jeans & Denim|
+----------

In [60]:
from pyspark.sql.functions import concat,concat_ws
product_product_tag_cloting_df=product_product_tag_cloting_df.select(concat_ws(' ',product_product_tag_cloting_df.name,product_product_tag_cloting_df.tag_name).alias('collection_tag'),'product_id')

In [61]:
product_product_tag_cloting_df.show()

+--------------------+----------+
|      collection_tag|product_id|
+--------------------+----------+
|    Swimwear bottoms|         1|
|      Skirts bottoms|         1|
|      Shorts bottoms|         1|
|       Pants bottoms|         1|
|Activewear & Loun...|         1|
|           Tops work|         1|
|           Tops work|         1|
|           Work work|         1|
|           Work work|         1|
|           Work work|         1|
|           Tops work|         1|
|      Occasions work|         1|
|         Black black|         1|
|         Black black|         1|
|         Black black|         1|
|         Black black|         1|
|         Black black|         1|
|         Pants black|         1|
|     Jumpsuits black|         1|
| Jeans & Denim black|         1|
+--------------------+----------+
only showing top 20 rows



In [62]:
data=(product_product_tag_cloting_df.groupby('product_id').pivot('collection_tag').count())

In [63]:
data.show()

+----------+----------------------------------+-------------------------------------------------+------------------------------------------+------------------------------------+-------------------------------+-------------------------------+--------------------------------+----------------------------------+---------------------------------+----------------------------------+-----------------------------------+----------------------------------+-------------+----------------+----------------+------------+--------------+-------------+------------+-----------+-----------+-----------------------------+---------------+---------------+-------------------+-----------------+-------------------------+---------------+-------------------+-----------------------------+------------------------+-------------------------------+----------------------------------+---------------------------+----------------------------------+---------------+-----------------------+------------------------+-----------+

In [64]:
data=data.na.fill(0)
data.show()

+----------+----------------------------------+-------------------------------------------------+------------------------------------------+------------------------------------+-------------------------------+-------------------------------+--------------------------------+----------------------------------+---------------------------------+----------------------------------+-----------------------------------+----------------------------------+-------------+----------------+----------------+------------+--------------+-------------+------------+-----------+-----------+-----------------------------+---------------+---------------+-------------------+-----------------+-------------------------+---------------+-------------------+-----------------------------+------------------------+-------------------------------+----------------------------------+---------------------------+----------------------------------+---------------+-----------------------+------------------------+-----------+

In [65]:
data_df=data.toPandas()
data_df=data_df.set_index('product_id')

In [66]:
data_df

,Activewear & Loungewear activewear,Activewear & Loungewear activewear-and-loungewear,Activewear & Loungewear activewear-jackets,Activewear & Loungewear biker-shorts,Activewear & Loungewear bottoms,Activewear & Loungewear hoodies,Activewear & Loungewear leggings,Activewear & Loungewear loungewear,Activewear & Loungewear sleepwear,Activewear & Loungewear sweatpants,...,Wedding Guest Categories bridesmaid,Wedding Guest Categories casual-wedding,Wedding Guest Categories cocktail,Wedding Guest Categories formal,Wedding Guest Categories semi-formal,Wedding Guest wedding-guest,White white,Work work,Wrap Dresses wrap-dresses,Wrap wrap-sweaters
product_id,,,,,,,,,,,,,,,,,,,,,
829,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,3,0,0,0,0
1436,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2162,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6731,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6107,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1522,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3380,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
data_df=string_column(data_df)

In [68]:
data_df.head()

,Activewear & Loungewear activewear,Activewear & Loungewear activewear-and-loungewear,Activewear & Loungewear activewear-jackets,Activewear & Loungewear biker-shorts,Activewear & Loungewear bottoms,Activewear & Loungewear hoodies,Activewear & Loungewear leggings,Activewear & Loungewear loungewear,Activewear & Loungewear sleepwear,Activewear & Loungewear sweatpants,...,Wedding Guest Categories bridesmaid,Wedding Guest Categories casual-wedding,Wedding Guest Categories cocktail,Wedding Guest Categories formal,Wedding Guest Categories semi-formal,Wedding Guest wedding-guest,White white,Work work,Wrap Dresses wrap-dresses,Wrap wrap-sweaters
product_id,,,,,,,,,,,,,,,,,,,,,
829,,,,,,,,,,,...,,Wedding Guest Categories casual-wedding,Wedding Guest Categories cocktail,Wedding Guest Categories formal,Wedding Guest Categories semi-formal,Wedding Guest wedding-guest,,,,
1436,,,,,Activewear & Loungewear bottoms,,,,,,...,,,,,,,,,,
3414,,,,,,,,,,,...,,,,,,,,,,
2162,,,,,,,,,,,...,,,,,,,,,,
6731,,,,,,,,,,,...,,,,,,,,,,


In [69]:
columns=data_df.columns
data_df['tag_string']=data_df[columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

C:\Users\Z2 mini\AppData\Local\Temp\ipykernel_8852\1634492.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_df['tag_string']=data_df[columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)


In [70]:
data_df.to_csv('D:/PROJECTS/FASHIONPASS/Database/product_cloth_cloection_tag_v2_10_1_2023.csv')